In [ ]:
import cv2
import math
import numpy as np
import keras_ocr
import os

def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2) / 2)
    y_mid = int((y1 + y2) / 2)
    return (x_mid, y_mid)

def inpaint_text(img_path, pipeline):  # Pass the pipeline as an argument
    img = keras_ocr.tools.read(img_path) 
    prediction_groups = pipeline.recognize([img])
    
    mask = np.zeros(img.shape[:2], dtype="uint8")
    for box in prediction_groups[0]:
        x0, y0 = box[1][0]
        x1, y1 = box[1][1] 
        x2, y2 = box[1][2]
        x3, y3 = box[1][3]
        
        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mid1 = midpoint(x0, y0, x3, y3)
        
        thickness = int(math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2))
        
        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mid1), 255, thickness)
    
    inpainted_img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
             
    return inpainted_img

def main():
    pipeline = keras_ocr.pipeline.Pipeline()


    input_dir = "D:\\Research Related Work\\large Dataset\\large dataset\\Xray disease\\Text Remove\\normal"# Directory containing input images
    output_dir =  "D:\\Research Related Work\\large Dataset\\large dataset\\Xray disease\\Text Remove\\NRR"# Directory to save preprocessed images

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith(".jpeg") or filename.endswith(".jpg") or filename.endswith(".png"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            
            inpainted_image = inpaint_text(input_path, pipeline)  # Pass the pipeline as an argument
            
            cv2.imwrite(output_path, inpainted_image)
            
            print(f"Processed: {filename}")

if __name__ == '__main__':
    main()
